In [36]:
# Fetch all product categories ID, display name, and number of products for each

import os
import xmlrpc.client
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

# Get environment variables
JUSTFRAMEIT_ODOO_V14_URL = os.getenv('JUSTFRAMEIT_ODOO_V14_URL')
JUSTFRAMEIT_ODOO_V14_DB = os.getenv('JUSTFRAMEIT_ODOO_V14_DB')
JUSTFRAMEIT_ODOO_V14_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_V14_USERNAME')
JUSTFRAMEIT_ODOO_V14_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_V14_API_KEY')

print("=" * 80)
print("Fetching Product Categories from Odoo V14")
print("=" * 80)

try:
    # Authenticate
    common = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_V14_URL}/xmlrpc/2/common", allow_none=True)
    uid = common.authenticate(JUSTFRAMEIT_ODOO_V14_DB, JUSTFRAMEIT_ODOO_V14_USERNAME, JUSTFRAMEIT_ODOO_V14_API_KEY, {})
    print(f"✅ Authenticated! UID: {uid}\n")
    
    # Create models proxy
    models = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_V14_URL}/xmlrpc/2/object", allow_none=True)
    
    # Fetch all product categories
    category_ids = models.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'product.category', 'search', [[]])
    
    # Read category details
    categories = models.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'product.category', 'read', [category_ids], {'fields': ['id', 'display_name']})
    
    print(f"Found {len(categories)} product categories:\n")
    
    # For each category, count the number of products
    for category in categories:
        # Count products in this category
        product_count = models.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid, JUSTFRAMEIT_ODOO_V14_API_KEY,
            'product.template', 'search_count', [[['categ_id', '=', category['id']]]])
        
        print(f"ID: {category['id']:4d} | Name: {category['display_name']:50s} | Products: {product_count:4d}")
    
except Exception as e:
    print(f"❌ Failed: {e}")

print("\n" + "=" * 80)
print("Category fetch complete!")
print("=" * 80)


Fetching Product Categories from Odoo V14
✅ Authenticated! UID: 28

Found 54 product categories:

ID:   62 | Name: 0. Custom                                          | Products:    0
ID:    1 | Name: 0. Presets                                         | Products: 1972
ID:    4 | Name: 0. Presets / 1. Presets zonder PP                  | Products:   13
ID:    8 | Name: 0. Presets / 2. Presets met PP                     | Products:    6
ID:   10 | Name: 0. Presets / 3. Losse verkopen                     | Products:    7
ID:   34 | Name: 0. Presets / 4. Diensten                           | Products:    0
ID:   57 | Name: 0. Presets / 4. Losse PP                           | Products:    1
ID:   78 | Name: 0. Presets / Expenses                              | Products:    0
ID:   82 | Name: 0. Presets / Fixed margin                          | Products:    1
ID:   77 | Name: 0. Presets / Saleable                              | Products:    0
ID:   76 | Name: 0. Presets Craft PP                

In [46]:
# Compare BOMs between Odoo V14 and V18 to find mismatches

import os
import xmlrpc.client
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

# Get environment variables for V14
JUSTFRAMEIT_ODOO_V14_URL = os.getenv('JUSTFRAMEIT_ODOO_V14_URL')
JUSTFRAMEIT_ODOO_V14_DB = os.getenv('JUSTFRAMEIT_ODOO_V14_DB')
JUSTFRAMEIT_ODOO_V14_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_V14_USERNAME')
JUSTFRAMEIT_ODOO_V14_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_V14_API_KEY')

# Get environment variables for V18
JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB')
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

print("=" * 80)
print("Comparing BOMs between Odoo V14 and V18")
print("=" * 80)

try:
    # Authenticate to V14
    print("\n[V14] Authenticating...")
    common_v14 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_V14_URL}/xmlrpc/2/common", allow_none=True)
    uid_v14 = common_v14.authenticate(JUSTFRAMEIT_ODOO_V14_DB, JUSTFRAMEIT_ODOO_V14_USERNAME, JUSTFRAMEIT_ODOO_V14_API_KEY, {})
    print(f"✅ V14 Authenticated! UID: {uid_v14}")
    
    # Authenticate to V18
    print("\n[V18] Authenticating...")
    common_v18 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common", allow_none=True)
    uid_v18 = common_v18.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})
    print(f"✅ V18 Authenticated! UID: {uid_v18}\n")
    
    # Create models proxies
    models_v14 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_V14_URL}/xmlrpc/2/object", allow_none=True)
    models_v18 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object", allow_none=True)
    
    # Define specific category IDs to check
    target_category_ids = [4, 8, 10, 57, 82]
    #target_category_ids = [10] #for testing
    
    print(f"Checking specific categories: {target_category_ids}\n")
    
    # Get category details from V14 (single call)
    categories = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'product.category', 'read',
        [target_category_ids],
        {'fields': ['display_name']})
    
    print("Categories to check:")
    for cat in categories:
        print(f"  - Category ID {cat['id']}: {cat['display_name']}")
    print()
    
    # Get all product templates in these categories from V14 (single call)
    print("[V14] Fetching product templates in these categories...")
    product_template_ids = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'product.template', 'search',
        [[['categ_id', 'in', target_category_ids]]])
    
    print(f"Found {len(product_template_ids)} product templates\n")
    
    # Search for BOMs with is_custom_made = False in V14 (single call)
    print("[V14] Searching for BOMs with is_custom_made = False...")
    print("=" * 80)
    
    bom_ids_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'mrp.bom', 'search',
        [[['product_tmpl_id', 'in', product_template_ids], ['is_custom_made', '=', False]]])
    
    print(f"\n[V14] Found {len(bom_ids_v14)} BOMs with is_custom_made = False\n")
    
    if not bom_ids_v14:
        print("No BOMs found with is_custom_made = False in V14")
    else:
        # Get ALL BOM details from V14 in a single call
        boms_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
            'mrp.bom', 'read',
            [bom_ids_v14],
            {'fields': ['id', 'code', 'display_name', 'product_tmpl_id', 'product_id', 'is_custom_made', 'bom_line_ids']})
        
        # Get ALL BOM lines from V14 in a single call
        all_bom_line_ids_v14 = []
        for bom in boms_v14:
            if bom['bom_line_ids']:
                all_bom_line_ids_v14.extend(bom['bom_line_ids'])
        
        bom_lines_v14_dict = {}
        if all_bom_line_ids_v14:
            all_bom_lines_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
                'mrp.bom.line', 'read',
                [all_bom_line_ids_v14],
                {'fields': ['id', 'product_id', 'product_qty', 'bom_id']})
            
            # Get all product IDs from BOM lines to fetch product codes
            product_ids_v14 = [line['product_id'][0] for line in all_bom_lines_v14 if line['product_id']]
            
            # Fetch product codes for all products in V14
            products_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
                'product.product', 'read',
                [product_ids_v14],
                {'fields': ['id', 'product_code']})
            
            # Create a mapping of product_id to product_code
            product_code_map_v14 = {p['id']: p['product_code'] for p in products_v14}
            
            # Organize by bom_id for easy lookup
            for line in all_bom_lines_v14:
                bom_id = line['bom_id'][0] if line['bom_id'] else None
                if bom_id not in bom_lines_v14_dict:
                    bom_lines_v14_dict[bom_id] = []
                # Add product_code to the line data
                product_id = line['product_id'][0] if line['product_id'] else None
                line['product_code'] = product_code_map_v14.get(product_id, 'N/A')
                bom_lines_v14_dict[bom_id].append(line)
        
        # Collect all BOM codes to search in V18
        bom_codes_v14 = [bom['code'] for bom in boms_v14 if bom['code']]
        
        # Search for ALL corresponding BOMs in V18 in a single call
        bom_ids_v18 = []
        if bom_codes_v14:
            bom_ids_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                'mrp.bom', 'search',
                [[['code', 'in', bom_codes_v14]]])
        
        # Get ALL BOM details from V18 in a single call
        boms_v18_dict = {}
        if bom_ids_v18:
            boms_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                'mrp.bom', 'read',
                [bom_ids_v18],
                {'fields': ['id', 'code', 'display_name', 'product_tmpl_id', 'product_id', 'bom_line_ids']})
            
            # Organize by code for easy lookup
            for bom in boms_v18:
                if bom['code']:
                    boms_v18_dict[bom['code']] = bom
            
            # Get ALL BOM lines from V18 in a single call
            all_bom_line_ids_v18 = []
            for bom in boms_v18:
                if bom['bom_line_ids']:
                    all_bom_line_ids_v18.extend(bom['bom_line_ids'])
            
            bom_lines_v18_dict = {}
            if all_bom_line_ids_v18:
                all_bom_lines_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                    'mrp.bom.line', 'read',
                    [all_bom_line_ids_v18],
                    {'fields': ['id', 'product_id', 'product_qty', 'bom_id']})
                
                # Get all product IDs from BOM lines to fetch product codes
                product_ids_v18 = [line['product_id'][0] for line in all_bom_lines_v18 if line['product_id']]
                
                # Fetch product codes for all products in V18
                products_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                    'product.product', 'read',
                    [product_ids_v18],
                    {'fields': ['id', 'x_studio_product_code']})
                
                # Create a mapping of product_id to x_studio_product_code
                product_code_map_v18 = {p['id']: p['x_studio_product_code'] for p in products_v18}
                
                # Organize by bom_id for easy lookup
                for line in all_bom_lines_v18:
                    bom_id = line['bom_id'][0] if line['bom_id'] else None
                    if bom_id not in bom_lines_v18_dict:
                        bom_lines_v18_dict[bom_id] = []
                    # Add product_code to the line data
                    product_id = line['product_id'][0] if line['product_id'] else None
                    line['product_code'] = product_code_map_v18.get(product_id, 'N/A')
                    bom_lines_v18_dict[bom_id].append(line)
        
        print("Comparing BOMs between V14 and V18:")
        print("-" * 80)
        
        mismatches_found = 0
        
        for bom_v14 in boms_v14:
            product_tmpl_name = bom_v14['product_tmpl_id'][1] if bom_v14['product_tmpl_id'] else 'N/A'
            product_variant_name = bom_v14['product_id'][1] if bom_v14['product_id'] else 'N/A (Template BOM)'
            bom_reference = bom_v14['code'] if bom_v14['code'] else 'No reference'
            bom_display_name = bom_v14['display_name'] if bom_v14['display_name'] else 'No display name'
            
            print(f"\n{'=' * 80}")
            print(f"BOM ID (V14): {bom_v14['id']}")
            print(f"  Reference: {bom_reference}")
            print(f"  Display Name: {bom_display_name}")
            print(f"  Product Template: {product_tmpl_name}")
            print(f"  Product Variant: {product_variant_name}")
            print(f"  is_custom_made (V14): {bom_v14['is_custom_made']}")
            
            # Get BOM line components from V14 (from pre-fetched data)
            components_v14 = {}
            bom_lines = bom_lines_v14_dict.get(bom_v14['id'], [])
            
            if bom_lines:
                print(f"\n  [V14] Components ({len(bom_lines)} items):")
                for line in bom_lines:
                    component_name = line['product_id'][1] if line['product_id'] else 'N/A'
                    component_code = line['product_code']
                    component_qty = line['product_qty']
                    components_v14[component_code] = {'name': component_name, 'qty': component_qty}
                    print(f"    - {component_name} (Code: {component_code}, Qty: {component_qty})")
            else:
                print(f"\n  [V14] Components: None")
            
            # Find corresponding BOM in V18 (from pre-fetched data)
            print(f"\n  [V18] Searching for corresponding BOM by matching BOM reference (code)...")
            
            if not bom_reference or bom_reference == 'No reference':
                print(f"  ❌ NO BOM REFERENCE (CODE) IN V14 - CANNOT MATCH!")
                mismatches_found += 1
                continue
            
            print(f"     BOM reference (V14): {bom_reference}")
            
            bom_v18 = boms_v18_dict.get(bom_reference)
            
            if not bom_v18:
                print(f"  ❌ NO CORRESPONDING BOM FOUND IN V18 (searched by code: '{bom_reference}')!")
                mismatches_found += 1
                continue
            
            print(f"  ✅ Found corresponding BOM in V18 (ID: {bom_v18['id']}, Code: '{bom_v18['code']}')")
            
            # Get BOM line components from V18 (from pre-fetched data)
            components_v18 = {}
            bom_lines_v18 = bom_lines_v18_dict.get(bom_v18['id'], [])
            
            if bom_lines_v18:
                print(f"\n  [V18] Components ({len(bom_lines_v18)} items):")
                for line in bom_lines_v18:
                    component_name = line['product_id'][1] if line['product_id'] else 'N/A'
                    component_code = line['product_code']
                    component_qty = line['product_qty']
                    components_v18[component_code] = {'name': component_name, 'qty': component_qty}
                    print(f"    - {component_name} (Code: {component_code}, Qty: {component_qty})")
            else:
                print(f"\n  [V18] Components: None")
            
            # Compare components
            print(f"\n  🔍 COMPARISON RESULTS:")
            has_mismatch = False
            
            # Check for missing components in V18
            for component_code, component_data in components_v14.items():
                if component_code not in components_v18:
                    print(f"    ❌ MISSING IN V18: {component_data['name']} (Code: {component_code}, Qty in V14: {component_data['qty']})")
                    has_mismatch = True
                elif components_v18[component_code]['qty'] != component_data['qty']:
                    print(f"    ⚠️  QUANTITY MISMATCH: {component_data['name']} (Code: {component_code})")
                    print(f"       V14 Qty: {component_data['qty']}")
                    print(f"       V18 Qty: {components_v18[component_code]['qty']}")
                    has_mismatch = True
            
            # Check for extra components in V18
            for component_code, component_data in components_v18.items():
                if component_code not in components_v14:
                    print(f"    ➕ EXTRA IN V18: {component_data['name']} (Code: {component_code}, Qty: {component_data['qty']})")
                    has_mismatch = True
            
            if not has_mismatch:
                print(f"    ✅ All components match!")
            else:
                mismatches_found += 1
    
    print("\n" + "=" * 80)
    print(f"BOM comparison complete!")
    print(f"Total BOMs checked: {len(bom_ids_v14) if bom_ids_v14 else 0}")
    print(f"BOMs with mismatches: {mismatches_found}")
    print("=" * 80)
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


Comparing BOMs between Odoo V14 and V18

[V14] Authenticating...
✅ V14 Authenticated! UID: 28

[V18] Authenticating...
✅ V18 Authenticated! UID: 2

Checking specific categories: [4, 8, 10, 57, 82]

Categories to check:
  - Category ID 4: 0. Presets / 1. Presets zonder PP
  - Category ID 8: 0. Presets / 2. Presets met PP
  - Category ID 10: 0. Presets / 3. Losse verkopen
  - Category ID 57: 0. Presets / 4. Losse PP
  - Category ID 82: 0. Presets / Fixed margin

[V14] Fetching product templates in these categories...
Found 28 product templates

[V14] Searching for BOMs with is_custom_made = False...

[V14] Found 249 BOMs with is_custom_made = False

Comparing BOMs between V14 and V18:
--------------------------------------------------------------------------------

BOM ID (V14): 53
  Reference: Los glas (Blinked plexi glas 2mm)
  Display Name: Los glas (Blinked plexi glas 2mm): Los glas
  Product Template: Los glas
  Product Variant: Los glas (Blinked plexi glas 2mm)
  is_custom_made (V1

In [64]:
# Compare material_cost (V14) vs x_studio_computed_cost (V18) for product templates

import os
import xmlrpc.client
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

# Get environment variables for V14
JUSTFRAMEIT_ODOO_V14_URL = os.getenv('JUSTFRAMEIT_ODOO_V14_URL')
JUSTFRAMEIT_ODOO_V14_DB = os.getenv('JUSTFRAMEIT_ODOO_V14_DB')
JUSTFRAMEIT_ODOO_V14_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_V14_USERNAME')
JUSTFRAMEIT_ODOO_V14_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_V14_API_KEY')

# Get environment variables for V18
JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB')
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

print("=" * 80)
print("Comparing material_cost (V14) vs x_studio_computed_cost (V18) for Product Templates")
print("=" * 80)

try:
    # Authenticate to V14
    print("\n[V14] Authenticating...")
    common_v14 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_V14_URL}/xmlrpc/2/common", allow_none=True)
    uid_v14 = common_v14.authenticate(JUSTFRAMEIT_ODOO_V14_DB, JUSTFRAMEIT_ODOO_V14_USERNAME, JUSTFRAMEIT_ODOO_V14_API_KEY, {})
    print(f"✅ V14 Authenticated! UID: {uid_v14}")
    
    # Authenticate to V18
    print("\n[V18] Authenticating...")
    common_v18 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common", allow_none=True)
    uid_v18 = common_v18.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})
    print(f"✅ V18 Authenticated! UID: {uid_v18}\n")
    
    # Create models proxies
    models_v14 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_V14_URL}/xmlrpc/2/object", allow_none=True)
    models_v18 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object", allow_none=True)
    
    # Define category IDs to EXCLUDE
    excluded_category_ids = [1, 4, 8, 10, 57, 82]
    
    print(f"Excluding categories: {excluded_category_ids}\n")
    
    # Get category details from V14
    categories = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'product.category', 'read',
        [excluded_category_ids],
        {'fields': ['display_name']})
    
    print("Categories to exclude:")
    for cat in categories:
        print(f"  - Category ID {cat['id']}: {cat['display_name']}")
    print()
    
    # Get all product templates from V14 excluding the specified categories
    print("[V14] Fetching product templates (excluding specified categories)...")
    template_ids_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'product.template', 'search',
        [[['categ_id', 'not in', excluded_category_ids]]])
    
    print(f"Found {len(template_ids_v14)} product templates in V14\n")
    
    # Get product template details from V14
    print("[V14] Reading product template details (id, product_code, material_cost, categ_id)...")
    templates_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'product.template', 'read',
        [template_ids_v14],
        {'fields': ['id', 'default_code', 'product_code', 'name', 'material_cost', 'categ_id']})
    
    print(f"Retrieved details for {len(templates_v14)} product templates\n")
    
    # Get all product codes from V14 (excluding None values)
    product_codes_v14 = [t['product_code'] for t in templates_v14 if t.get('product_code')]
    
    print(f"Found {len(product_codes_v14)} product templates with product_code in V14\n")
    
    # Fetch ALL matching product templates from V18 in ONE API call using product codes
    print("[V18] Fetching all matching product templates by x_studio_product_code in one call...")
    template_ids_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
        'product.template', 'search',
        [[['x_studio_product_code', 'in', product_codes_v14]]])
    
    print(f"Found {len(template_ids_v18)} matching product templates in V18\n")
    
    # Get all V18 product template details in ONE API call
    print("[V18] Reading all product template details in one call...")
    templates_v18_list = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
        'product.template', 'read',
        [template_ids_v18],
        {'fields': ['id', 'default_code', 'x_studio_product_code', 'name', 'x_studio_computed_cost']})
    
    # Create a dictionary for quick lookup by x_studio_product_code
    templates_v18_dict = {t['x_studio_product_code']: t for t in templates_v18_list if t.get('x_studio_product_code')}
    
    print(f"Retrieved details for {len(templates_v18_list)} product templates from V18\n")
    
    print("Comparing costs between V14 and V18:")
    print("-" * 80)
    
    mismatches_found = 0
    matches_found = 0
    templates_checked = 0
    templates_not_found_in_v18 = 0
    templates_without_product_code = 0
    
    for template_v14 in templates_v14:
        template_id = template_v14['id']
        template_code = template_v14['default_code']
        product_code_v14 = template_v14.get('product_code')
        template_name = template_v14['name']
        material_cost_v14 = template_v14['material_cost'] if template_v14['material_cost'] else 0.0
        category_name = template_v14['categ_id'][1] if template_v14['categ_id'] else 'N/A'
        
        # Skip if no product_code in V14
        if not product_code_v14:
            templates_without_product_code += 1
            continue
        
        templates_checked += 1
        
        # Look up product template in V18 dictionary by product_code
        template_v18 = templates_v18_dict.get(product_code_v14)
        
        if not template_v18:
            print(f"\n❌ Product Template NOT FOUND in V18:")
            print(f"   ID (V14): {template_id}")
            print(f"   Code: {template_code}")
            print(f"   product_code (V14): {product_code_v14}")
            print(f"   Name: {template_name}")
            print(f"   Category: {category_name}")
            print(f"   material_cost (V14): €{material_cost_v14:.2f}")
            templates_not_found_in_v18 += 1
            continue
        
        computed_cost_v18 = template_v18['x_studio_computed_cost'] if template_v18['x_studio_computed_cost'] else 0.0
        
        # Compare costs (with small tolerance for floating point comparison)
        tolerance = 0.01
        if abs(material_cost_v14 - computed_cost_v18) > tolerance:
            print(f"\n⚠️  COST MISMATCH:")
            print(f"   ID (V14): {template_id}")
            print(f"   ID (V18): {template_v18['id']}")
            print(f"   Code: {template_code}")
            print(f"   product_code (V14): {product_code_v14}")
            print(f"   x_studio_product_code (V18): {template_v18['x_studio_product_code']}")
            print(f"   Name (V14): {template_name}")
            print(f"   Name (V18): {template_v18['name']}")
            print(f"   Category: {category_name}")
            print(f"   material_cost (V14): €{material_cost_v14:.2f}")
            print(f"   x_studio_computed_cost (V18): €{computed_cost_v18:.2f}")
            print(f"   Difference: €{abs(material_cost_v14 - computed_cost_v18):.2f}")
            mismatches_found += 1
        else:
            matches_found += 1
    
    print("\n" + "=" * 80)
    print(f"Cost comparison complete!")
    print(f"Total product templates in V14 (excluding categories): {len(templates_v14)}")
    print(f"Product templates without product_code (skipped): {templates_without_product_code}")
    print(f"Product templates checked: {templates_checked}")
    print(f"Product templates not found in V18: {templates_not_found_in_v18}")
    print(f"Product templates with cost matches: {matches_found}")
    print(f"Product templates with cost mismatches: {mismatches_found}")
    print("=" * 80)
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


Comparing material_cost (V14) vs x_studio_computed_cost (V18) for Product Templates

[V14] Authenticating...
✅ V14 Authenticated! UID: 28

[V18] Authenticating...
✅ V18 Authenticated! UID: 2

Excluding categories: [1, 4, 8, 10, 57, 82]

Categories to exclude:
  - Category ID 1: 0. Presets
  - Category ID 4: 0. Presets / 1. Presets zonder PP
  - Category ID 8: 0. Presets / 2. Presets met PP
  - Category ID 10: 0. Presets / 3. Losse verkopen
  - Category ID 57: 0. Presets / 4. Losse PP
  - Category ID 82: 0. Presets / Fixed margin

[V14] Fetching product templates (excluding specified categories)...
Found 3318 product templates in V14

[V14] Reading product template details (id, product_code, material_cost, categ_id)...
Retrieved details for 3318 product templates

Found 3292 product templates with product_code in V14

[V18] Fetching all matching product templates by x_studio_product_code in one call...
Found 3234 matching product templates in V18

[V18] Reading all product template deta

In [55]:
# Sync seller_ids from V14 to V18 for ALL products that have more than 1 seller
# This will process all products with multiple sellers in V14

import os
import xmlrpc.client
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

# Get environment variables for V14
JUSTFRAMEIT_ODOO_V14_URL = os.getenv('JUSTFRAMEIT_ODOO_V14_URL')
JUSTFRAMEIT_ODOO_V14_DB = os.getenv('JUSTFRAMEIT_ODOO_V14_DB')
JUSTFRAMEIT_ODOO_V14_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_V14_USERNAME')
JUSTFRAMEIT_ODOO_V14_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_V14_API_KEY')

# Get environment variables for V18
JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB')
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

print("=" * 80)
print(f"Syncing seller_ids from V14 to V18 for ALL products with more than 1 seller")
print("=" * 80)

try:
    # Authenticate to V14
    print("\n[V14] Authenticating...")
    common_v14 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_V14_URL}/xmlrpc/2/common", allow_none=True)
    uid_v14 = common_v14.authenticate(JUSTFRAMEIT_ODOO_V14_DB, JUSTFRAMEIT_ODOO_V14_USERNAME, JUSTFRAMEIT_ODOO_V14_API_KEY, {})
    print(f"✅ V14 Authenticated! UID: {uid_v14}\n")
    
    # Create models proxy for V14
    models_v14 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_V14_URL}/xmlrpc/2/object", allow_none=True)
    
    # Get all product templates with seller_ids field from V14
    print(f"[V14] Fetching all product.template records with seller_ids...")
    all_product_ids = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'product.template', 'search',
        [[]])
    
    print(f"[V14] Found {len(all_product_ids)} total product templates")
    print(f"[V14] Reading seller_ids for all products...")
    
    all_products = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'product.template', 'read',
        [all_product_ids],
        {'fields': ['id', 'name', 'default_code', 'product_code', 'seller_ids']})
    
    # Filter products with more than 1 seller
    products_with_multiple_sellers = []
    for product in all_products:
        if product['seller_ids'] and len(product['seller_ids']) > 1:
            products_with_multiple_sellers.append(product)
    
    print(f"\n✅ Found {len(products_with_multiple_sellers)} products with more than 1 seller\n")
    
    if not products_with_multiple_sellers:
        print("⚠️  No products with multiple sellers found. Nothing to sync.")
        raise Exception("No products to sync")
    
    # Now authenticate to V18
    print("=" * 80)
    print("[V18] Authenticating...")
    common_v18 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common", allow_none=True)
    uid_v18 = common_v18.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})
    print(f"✅ V18 Authenticated! UID: {uid_v18}\n")
    
    # Create models proxy for V18
    models_v18 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object", allow_none=True)
    
    # Process each product
    print("=" * 80)
    print("Starting sync process...")
    print("=" * 80)
    
    total_products = len(products_with_multiple_sellers)
    synced_count = 0
    skipped_count = 0
    error_count = 0
    
    for product_index, template_v14 in enumerate(products_with_multiple_sellers, 1):
        print(f"\n{'=' * 80}")
        print(f"Processing product {product_index}/{total_products}")
        print(f"{'=' * 80}")
        
        product_code_v14 = template_v14.get('product_code')
        
        print(f"\n✅ Product Template (V14):")
        print(f"   ID: {template_v14['id']}")
        print(f"   Name: {template_v14['name']}")
        print(f"   Default Code: {template_v14['default_code']}")
        print(f"   Product Code: {product_code_v14}")
        print(f"   Number of Sellers: {len(template_v14['seller_ids'])}")
        
        if not product_code_v14:
            print("\n⚠️  No product_code found for this product template in V14. Skipping.")
            skipped_count += 1
            continue
        
        try:
            # Fetch seller details from V14
            print(f"\n[V14] Fetching details for {len(template_v14['seller_ids'])} seller records...")
            seller_info_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
                'product.supplierinfo', 'read',
                [template_v14['seller_ids']],
                {'fields': ['id', 'name', 'price', 'is_preferred_supplier', 'sequence', 'product_code']})
            
            print(f"\nSeller Information V14:")
            for idx, seller in enumerate(seller_info_v14, 1):
                print(f"   Seller #{idx}: {seller['name'][1]} (Price: {seller['price']}, Preferred: {seller['is_preferred_supplier']})")
            
            # Find matching product in V18 using x_studio_product_code
            print(f"\n[V18] Searching for product with x_studio_product_code = {product_code_v14}...")
            product_ids_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                'product.template', 'search',
                [[['x_studio_product_code', '=', product_code_v14]]])
            
            if not product_ids_v18:
                print(f"⚠️  No product found in V18 with x_studio_product_code = {product_code_v14}. Skipping.")
                skipped_count += 1
                continue
            
            product_id_v18 = product_ids_v18[0]
            print(f"✅ Found matching product in V18: ID {product_id_v18}")
            
            # Get product template from V18
            product_template_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                'product.template', 'read',
                [product_id_v18],
                {'fields': ['id', 'name', 'default_code', 'x_studio_product_code', 'seller_ids']})
            
            template_v18 = product_template_v18[0]
            print(f"\n✅ Product Template (V18):")
            print(f"   ID: {template_v18['id']}")
            print(f"   Name: {template_v18['name']}")
            print(f"   Current Seller IDs: {template_v18['seller_ids']}")
            
            # Get existing sellers in V18
            existing_sellers_v18 = []
            if template_v18['seller_ids']:
                existing_sellers_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                    'product.supplierinfo', 'read',
                    [template_v18['seller_ids']],
                    {'fields': ['id', 'partner_id', 'price', 'sequence']})
                
                print(f"\nExisting sellers in V18:")
                for seller in existing_sellers_v18:
                    print(f"   - {seller['partner_id'][1]} (Price: {seller['price']}, Sequence: {seller['sequence']})")
            
            # Process each seller from V14
            print(f"\nSyncing sellers...")
            
            # Sort sellers: preferred first, then by original sequence
            sorted_sellers_v14 = sorted(seller_info_v14, 
                                       key=lambda x: (not x['is_preferred_supplier'], x['sequence']))
            
            # Assign new sequences starting from 1
            for new_sequence, seller_v14 in enumerate(sorted_sellers_v14, start=1):
                supplier_name_v14 = seller_v14['name'][1]  # name is [id, name]
                supplier_id_v14 = seller_v14['name'][0]
                price_v14 = seller_v14['price']
                is_preferred_v14 = seller_v14['is_preferred_supplier']
                
                print(f"   Processing: {supplier_name_v14} (Price: {price_v14}, Sequence: {new_sequence}, Preferred: {is_preferred_v14})")
                
                # Search for partner in V18 by name
                partner_ids_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                    'res.partner', 'search',
                    [[['name', '=', supplier_name_v14]]])
                
                if not partner_ids_v18:
                    print(f"      Creating partner '{supplier_name_v14}' in V18...")
                    partner_id_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                        'res.partner', 'create',
                        [{'name': supplier_name_v14, 'supplier_rank': 1}])
                    print(f"      ✅ Created partner with ID: {partner_id_v18}")
                else:
                    partner_id_v18 = partner_ids_v18[0]
                
                # Check if this supplier already exists for this product in V18
                existing_seller = None
                for seller in existing_sellers_v18:
                    if seller['partner_id'][0] == partner_id_v18:
                        existing_seller = seller
                        break
                
                if existing_seller:
                    # Update existing seller
                    models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                        'product.supplierinfo', 'write',
                        [[existing_seller['id']], {
                            'price': price_v14,
                            'sequence': new_sequence
                        }])
                    print(f"      ✅ Updated existing seller (ID: {existing_seller['id']})")
                else:
                    # Create new seller
                    new_seller_id = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                        'product.supplierinfo', 'create',
                        [{
                            'partner_id': partner_id_v18,
                            'product_tmpl_id': product_id_v18,
                            'price': price_v14,
                            'sequence': new_sequence
                        }])
                    print(f"      ✅ Created new seller (ID: {new_seller_id})")
            
            synced_count += 1
            print(f"\n✅ Successfully synced product {template_v14['name']}")
            
        except Exception as e:
            error_count += 1
            print(f"\n❌ Error processing product {template_v14['name']}: {e}")
            import traceback
            traceback.print_exc()
            continue
    
    print("\n" + "=" * 80)
    print("Sync complete!")
    print("=" * 80)
    print(f"\nSummary:")
    print(f"   Total products with multiple sellers: {total_products}")
    print(f"   Successfully synced: {synced_count}")
    print(f"   Skipped: {skipped_count}")
    print(f"   Errors: {error_count}")
    print("=" * 80)
    
except Exception as e:
    print(f"\n❌ Fatal Error: {e}")
    import traceback
    traceback.print_exc()


Syncing seller_ids from V14 to V18 for ALL products with more than 1 seller

[V14] Authenticating...
✅ V14 Authenticated! UID: 28

[V14] Fetching all product.template records with seller_ids...
[V14] Found 5320 total product templates
[V14] Reading seller_ids for all products...

✅ Found 152 products with more than 1 seller

[V18] Authenticating...
✅ V18 Authenticated! UID: 2

Starting sync process...

Processing product 1/152

✅ Product Template (V14):
   ID: 5237
   Name: "Altis" Baklijst Eik Fineer 38X25
   Default Code: 100767.D08+D10
   Product Code: 100767
   Number of Sellers: 2

[V14] Fetching details for 2 seller records...

Seller Information V14:
   Seller #1: Eurolijsten (Price: 7.7, Preferred: True)
   Seller #2: Molgra (Price: 5.6, Preferred: False)

[V18] Searching for product with x_studio_product_code = 100767...
✅ Found matching product in V18: ID 9897

✅ Product Template (V18):
   ID: 9897
   Name: "Altis" Baklijst Eik Fineer 38X25
   Current Seller IDs: [5349, 7121]

In [63]:
# Update seller_ids sequence for products with specific x_studio_variable_a_label values in V18
# This temporarily changes the sequence to trigger automated actions

import os
import xmlrpc.client
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

# Get environment variables for V18
JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB')
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

# Define the target variable_a_label values
TARGET_VALUES = ['aankoopprijs', 'aankoopprijs_m2', 'leveranciersprijs', 'materiaalkost', 'price', 'prijs_papier']

print("=" * 80)
print(f"Updating seller_ids sequence for products with specific x_studio_variable_a_label values in V18")
print("=" * 80)
print(f"Target values: {', '.join(TARGET_VALUES)}")
print("=" * 80)

try:
    # Authenticate to V18
    print("\n[V18] Authenticating...")
    common_v18 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common", allow_none=True)
    uid_v18 = common_v18.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})
    print(f"✅ V18 Authenticated! UID: {uid_v18}\n")
    
    # Create models proxy for V18
    models_v18 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object", allow_none=True)
    
    # Get all product templates with matching x_studio_variable_a_label values
    print(f"[V18] Fetching product.template records with matching x_studio_variable_a_label values...")
    product_templates = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
        'product.template', 'search_read',
        [[['x_studio_variable_a_label', 'in', TARGET_VALUES]]],
        {'fields': ['id', 'name', 'default_code', 'x_studio_variable_a_label', 'seller_ids']})
    
    print(f"✅ Found {len(product_templates)} product templates matching criteria\n")
    
    if not product_templates:
        print(f"❌ No products found with the specified x_studio_variable_a_label values!")
    else:
        processed_count = 0
        updated_count = 0
        skipped_count = 0
        error_count = 0
        
        for product in product_templates:
            try:
                processed_count += 1
                print(f"\n[{processed_count}/{len(product_templates)}] Processing product:")
                print(f"   ID: {product['id']}")
                print(f"   Name: {product['name']}")
                print(f"   Code: {product['default_code']}")
                print(f"   Variable A Label: {product['x_studio_variable_a_label']}")
                print(f"   Seller IDs: {product['seller_ids']}")
                
                if not product['seller_ids']:
                    print(f"   ⚠️  No seller_ids found - skipping")
                    skipped_count += 1
                    continue
                
                # Get detailed seller information
                sellers = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                    'product.supplierinfo', 'search_read',
                    [[['id', 'in', product['seller_ids']]]],
                    {'fields': ['id', 'partner_id', 'price', 'sequence']})
                
                print(f"   Current sellers ({len(sellers)}):")
                for seller in sellers:
                    print(f"      - ID: {seller['id']}, Partner: {seller['partner_id'][1]}, Price: {seller['price']}, Sequence: {seller['sequence']}")
                
                # Find the first seller with sequence 1
                seller_with_seq_1 = next((s for s in sellers if s['sequence'] == 1), None)
                
                if seller_with_seq_1:
                    print(f"   Found seller with sequence 1: ID {seller_with_seq_1['id']}")
                    
                    # Step 1: Change sequence from 1 to 2
                    print(f"      Step 1: Changing sequence from 1 to 2...")
                    models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                        'product.supplierinfo', 'write',
                        [[seller_with_seq_1['id']], {'sequence': 2}])
                    print(f"      ✅ Changed sequence to 2")
                    
                    # Step 2: Change sequence back from 2 to 1
                    print(f"      Step 2: Changing sequence back from 2 to 1...")
                    models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                        'product.supplierinfo', 'write',
                        [[seller_with_seq_1['id']], {'sequence': 1}])
                    print(f"      ✅ Changed sequence back to 1")
                    
                    updated_count += 1
                    print(f"   ✅ Successfully triggered automated action for product {product['id']}")
                else:
                    print(f"   ⚠️  No seller with sequence 1 found - skipping")
                    skipped_count += 1
                
            except Exception as e:
                error_count += 1
                print(f"   ❌ Error processing product {product['id']}: {e}")
                import traceback
                traceback.print_exc()
                continue
        
        print(f"\n{'=' * 80}")
        print(f"Update complete!")
        print(f"{'=' * 80}")
        print(f"Summary:")
        print(f"   Total products found: {len(product_templates)}")
        print(f"   Successfully updated: {updated_count}")
        print(f"   Skipped (no sellers or no sequence 1): {skipped_count}")
        print(f"   Errors: {error_count}")
        print("=" * 80)
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


Updating seller_ids sequence for products with specific x_studio_variable_a_label values in V18
Target values: aankoopprijs, aankoopprijs_m2, leveranciersprijs, materiaalkost, price, prijs_papier

[V18] Authenticating...
✅ V18 Authenticated! UID: 2

[V18] Fetching product.template records with matching x_studio_variable_a_label values...
✅ Found 227 product templates matching criteria


[1/227] Processing product:
   ID: 13261
   Name: Alabaster
   Code: 4825.C2
   Variable A Label: aankoopprijs_m2
   Seller IDs: [7144, 7143]
   Current sellers (2):
      - ID: 7144, Partner: Tfc, Price: 6.5, Sequence: 1
      - ID: 7143, Partner: Nielsen Design, Price: 5.3, Sequence: 2
   Found seller with sequence 1: ID 7144
      Step 1: Changing sequence from 1 to 2...
      ✅ Changed sequence to 2
      Step 2: Changing sequence back from 2 to 1...
      ✅ Changed sequence back to 1
   ✅ Successfully triggered automated action for product 13261

[2/227] Processing product:
   ID: 13262
   Name: Al